In [1]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm
from copy import deepcopy

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

In [2]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

# Import Data

In [3]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

pd.options.display.max_columns = None
pd.options.display.max_rows = 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import Data

In [4]:
dfRes = pd.read_csv("OUTPUT/MP/05-classifiers/DataV2/CLFRunCombined_SelCol.csv")

In [5]:
dfRes

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV,brt_md,brt_params
0,T81,T81_Holdout,2024-04-27 21:48:30.513965,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.845993,0.830796,0.967042,0.799135,0.685160,0.685160,0.976432,0.959327,0.792884,0.959327,NaN,T1000_itrr500_th100_topR5
1,T81,T81_Holdout,2024-04-27 21:48:30.513965,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.849266,0.834664,0.969056,0.804437,0.694857,0.694857,0.974471,0.956449,0.797947,0.956449,NaN,T1000_itrr500_th100_topR5
2,T81,T81_Holdout,2024-04-27 21:48:30.513965,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,0.885006,0.876654,0.971158,0.860965,0.796562,0.796562,0.956746,0.937275,0.853209,0.937275,NaN,T1000_itrr500_th100_topR5
3,T81,T81_Holdout,2024-04-27 21:48:30.513965,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),6.0,1000.0,42.0,NaN,NaN,0.891533,0.883766,0.972673,0.869714,0.808786,0.808786,0.958746,0.940969,0.860844,0.940969,NaN,T1000_itrr500_th100_topR5
4,T81,T81_Holdout,2024-04-27 21:48:30.513965,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),10.0,100.0,42.0,NaN,NaN,0.927315,0.924848,0.972825,0.917351,0.900793,0.900793,0.948903,0.934733,0.921872,0.934733,NaN,T1000_itrr500_th100_topR5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,DT,DT_Boruta_Top,2024-04-27 22:13:31.449763,XGBClassifier,Boruta_Top,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.915004,0.913629,0.953140,0.907525,0.889622,0.889622,0.937636,0.927877,0.906639,0.927877,7.0,T1000_itrr500_th100_topR5
980,DT,DT_Boruta_Top,2024-04-27 22:13:31.449763,XGBClassifier,Boruta_Top,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,100.0,42.0,NaN,1.0,0.928953,0.927411,0.963930,0.922159,0.900432,0.900432,0.954389,0.947599,0.917091,0.947599,7.0,T1000_itrr500_th100_topR5
981,DT,DT_Boruta_Top,2024-04-27 22:13:31.449763,XGBClassifier,Boruta_Top,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.916270,0.914834,0.953706,0.908736,0.889622,0.889622,0.940046,0.930293,0.906774,0.930293,7.0,T1000_itrr500_th100_topR5
982,DT,DT_Boruta_Top,2024-04-27 22:13:31.449763,XGBClassifier,Boruta_Top,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,1.0,0.921350,0.919950,0.965869,0.913946,0.895063,0.895063,0.944836,0.936998,0.912854,0.936998,7.0,T1000_itrr500_th100_topR5


In [6]:
dfRes.columns = [x.replace('mean_test_','') for x in dfRes.columns]

In [7]:
s_brt = 'and (RunType=="Boruta_True" or RunType=="Boruta_Top")'
s_RF10 = 'classifier=="RandomForestClassifier" and param_clf__n_estimators==1000 and param_clf__max_depth==10'
s_RFXGB10 = 'param_clf__n_estimators==1000 and param_clf__max_depth==10'

In [8]:
lPrettyCols =     [
        "MainDataset",
        "RunType",
        "classifier",
        "brt_nTrue",
        "brt_nTop",
        "accuracy",
        "balanced_accuracy",
        "roc_auc",
        "f1",
        "recall",
        "sensitivity",
        "specificity",
        "precision",
        "NPV",
        "PPV",
        "brt_md",
    ]
lPrettyColsSite = [
    "MainDataset",
    "RunType",
    'Site',
    "classifier",
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    "NPV",
    "PPV",
]

# Inspect Original Holdout

In [12]:
dfRes.query(f'{s_RF10} and RunType=="Holdout"').sort_values("balanced_accuracy", ascending=False)[lPrettyCols].T

,661,5,333
MainDataset,DT,T81,T85
RunType,Holdout,Holdout,Holdout
classifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier
brt_nTrue,NaN,NaN,NaN
brt_nTop,NaN,NaN,NaN
accuracy,0.952995,0.931675,0.673549
balanced_accuracy,0.951131,0.929258,0.617958
roc_auc,0.990039,0.974317,0.710181
f1,0.948538,0.922286,0.449992
recall,0.916721,0.905671,0.334993


In [ ]:
s_query = f'MainDataset=="T81" and param_clf__n_estimators==1000 and param_clf__max_depth==10 {s_brt}'
dfTmp = deepcopy(dfRes.query(s_query)).sort_values('balanced_accuracy',ascending=False)
print(s_query)
print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")
# print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")

dfTmp[lPrettyCols]

MainDataset=="T81" and param_clf__n_estimators==1000 and param_clf__max_depth==10 and (RunType=="Boruta_True" or RunType=="Boruta_Top")
Max balanced accuracy 0.9346908208996904
Max balanced accuracy 0.9346908208996904


,MainDataset,RunType,classifier,brt_nTrue,brt_nTop,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md
169,T81,Boruta_True,RandomForestClassifier,26.0,30.0,0.937009,0.934691,0.968085,0.928179,0.912577,0.912577,0.956805,0.944785,0.931586,0.944785,5.0
251,T81,Boruta_True,RandomForestClassifier,25.0,29.0,0.937009,0.934691,0.967777,0.928179,0.912577,0.912577,0.956805,0.944785,0.931586,0.944785,7.0
210,T81,Boruta_Top,RandomForestClassifier,26.0,30.0,0.937015,0.934681,0.968502,0.928258,0.912577,0.912577,0.956785,0.944772,0.931428,0.944772,5.0
87,T81,Boruta_True,RandomForestClassifier,29.0,34.0,0.935917,0.933471,0.970090,0.926729,0.910138,0.910138,0.956805,0.944690,0.930082,0.944690,3.0
292,T81,Boruta_Top,RandomForestClassifier,25.0,29.0,0.935922,0.933022,0.969853,0.926415,0.905319,0.905319,0.960726,0.949371,0.926811,0.949371,7.0
128,T81,Boruta_Top,RandomForestClassifier,29.0,34.0,0.932662,0.929818,0.969575,0.922771,0.902850,0.902850,0.956785,0.944158,0.924656,0.944158,3.0
139,T81,Boruta_Top,XGBClassifier,29.0,34.0,0.926146,0.922540,0.961511,0.914641,0.888275,0.888275,0.956805,0.944143,0.914665,0.944143,3.0
149,T81,Boruta_Top,XGBClassifier,29.0,34.0,0.923960,0.920101,0.958472,0.911684,0.883397,0.883397,0.956805,0.943478,0.911513,0.943478,3.0
303,T81,Boruta_Top,XGBClassifier,25.0,29.0,0.919630,0.916418,0.954665,0.907801,0.885836,0.885836,0.947001,0.931853,0.911655,0.931853,7.0
272,T81,Boruta_True,XGBClassifier,25.0,29.0,0.919624,0.916194,0.952037,0.907482,0.883426,0.883426,0.948961,0.933728,0.910132,0.933728,7.0


In [ ]:
# dfTmp.query('classifier=="RandomForestClassifier" and (RunType=="Boruta_True" or RunType=="Boruta_Top")')
dfTmp.query(
    'classifier=="RandomForestClassifier"'
)

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
169,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_True,5,26.0,30.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.937009,0.934691,0.968085,0.928179,0.912577,0.912577,0.956805,0.944785,0.931586,0.944785,5.0,T1000_itrr500_th100_topR5
251,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_True,5,25.0,29.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.937009,0.934691,0.967777,0.928179,0.912577,0.912577,0.956805,0.944785,0.931586,0.944785,7.0,T1000_itrr500_th100_topR5
210,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_Top,5,26.0,30.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.937015,0.934681,0.968502,0.928258,0.912577,0.912577,0.956785,0.944772,0.931428,0.944772,5.0,T1000_itrr500_th100_topR5
87,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_True,5,29.0,34.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.935917,0.933471,0.970090,0.926729,0.910138,0.910138,0.956805,0.944690,0.930082,0.944690,3.0,T1000_itrr500_th100_topR5
292,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_Top,5,25.0,29.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.935922,0.933022,0.969853,0.926415,0.905319,0.905319,0.960726,0.949371,0.926811,0.949371,7.0,T1000_itrr500_th100_topR5
128,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_Top,5,29.0,34.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.932662,0.929818,0.969575,0.922771,0.902850,0.902850,0.956785,0.944158,0.924656,0.944158,3.0,T1000_itrr500_th100_topR5


In [ ]:
dfTmp.query('classifier=="XGBClassifier"')

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
139,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_Top,5,29.0,34.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.926146,0.922540,0.961511,0.914641,0.888275,0.888275,0.956805,0.944143,0.914665,0.944143,3.0,T1000_itrr500_th100_topR5
149,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_Top,5,29.0,34.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.923960,0.920101,0.958472,0.911684,0.883397,0.883397,0.956805,0.943478,0.911513,0.943478,3.0,T1000_itrr500_th100_topR5
303,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_Top,5,25.0,29.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.919630,0.916418,0.954665,0.907801,0.885836,0.885836,0.947001,0.931853,0.911655,0.931853,7.0,T1000_itrr500_th100_topR5
272,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,25.0,29.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.919624,0.916194,0.952037,0.907482,0.883426,0.883426,0.948961,0.933728,0.910132,0.933728,7.0,T1000_itrr500_th100_topR5
262,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,25.0,29.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.919630,0.915969,0.956227,0.907317,0.881017,0.881017,0.950922,0.935892,0.908450,0.935892,7.0,T1000_itrr500_th100_topR5
98,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,29.0,34.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.919630,0.915725,0.957389,0.906942,0.878548,0.878548,0.952902,0.938507,0.907318,0.938507,3.0,T1000_itrr500_th100_topR5
108,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,29.0,34.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.918537,0.914511,0.953515,0.905444,0.876139,0.876139,0.952883,0.938079,0.905744,0.938079,3.0,T1000_itrr500_th100_topR5
180,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,26.0,30.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.916363,0.913243,0.955692,0.904225,0.883426,0.883426,0.943060,0.926785,0.909444,0.926785,5.0,T1000_itrr500_th100_topR5
313,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_Top,5,25.0,29.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.916375,0.912814,0.951718,0.903577,0.878607,0.878607,0.947020,0.931722,0.906948,0.931722,7.0,T1000_itrr500_th100_topR5
190,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,26.0,30.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.916357,0.912784,0.952876,0.903537,0.878548,0.878548,0.947020,0.931953,0.907021,0.931953,5.0,T1000_itrr500_th100_topR5


In [ ]:
dfTmp.crosstab('RunType','mean_test_balanced_accuracy')

AttributeError: 'DataFrame' object has no attribute 'crosstab'

# Inspect T81 Dataset

In [10]:
dfRes.query('MainDataset=="T81"').sort_values("balanced_accuracy", ascending=False)

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
56,T81,T81_Full,2024-04-27 21:48:30.513965,XGBClassifier,Full,5,NaN,NaN,"XGBClassifier(base_score=None, booster=None, c...",10.0,100.0,42.0,NaN,0.1,0.945394,0.944490,0.979135,0.938908,0.935681,0.935681,0.953299,0.942278,0.948056,0.942278,NaN,T1000_itrr500_th100_topR5
253,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_True,5,25.0,29.0,RandomForestClassifier(),15.0,1000.0,42.0,NaN,NaN,0.945693,0.944134,0.968296,0.938450,0.929503,0.929503,0.958765,0.948016,0.944416,0.948016,7.0,T1000_itrr500_th100_topR5
58,T81,T81_Full,2024-04-27 21:48:30.513965,XGBClassifier,Full,5,NaN,NaN,"XGBClassifier(base_score=None, booster=None, c...",15.0,100.0,42.0,NaN,0.1,0.944950,0.944086,0.979170,0.938455,0.935680,0.935680,0.952492,0.941357,0.947991,0.941357,NaN,T1000_itrr500_th100_topR5
50,T81,T81_Full,2024-04-27 21:48:30.513965,RandomForestClassifier,Full,5,NaN,NaN,RandomForestClassifier(),NaN,1000.0,42.0,NaN,NaN,0.943174,0.943439,0.974628,0.937260,0.946063,0.946063,0.940814,0.928719,0.955501,0.928719,NaN,T1000_itrr500_th100_topR5
170,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_True,5,26.0,30.0,RandomForestClassifier(),15.0,100.0,42.0,NaN,NaN,0.944606,0.942930,0.970362,0.937147,0.927094,0.927094,0.958765,0.947921,0.942621,0.947921,5.0,T1000_itrr500_th100_topR5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_True,5,25.0,29.0,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.876193,0.864882,0.964022,0.844727,0.757273,0.757273,0.972491,0.957387,0.832874,0.957387,7.0,T1000_itrr500_th100_topR5
1,T81,T81_Holdout,2024-04-27 21:48:30.513965,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.849266,0.834664,0.969056,0.804437,0.694857,0.694857,0.974471,0.956449,0.797947,0.956449,NaN,T1000_itrr500_th100_topR5
42,T81,T81_Full,2024-04-27 21:48:30.513965,RandomForestClassifier,Full,5,NaN,NaN,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.845949,0.830883,0.973655,0.799378,0.684310,0.684310,0.977455,0.961097,0.791947,0.961097,NaN,T1000_itrr500_th100_topR5
0,T81,T81_Holdout,2024-04-27 21:48:30.513965,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.845993,0.830796,0.967042,0.799135,0.685160,0.685160,0.976432,0.959327,0.792884,0.959327,NaN,T1000_itrr500_th100_topR5


In [28]:
s_query = f'MainDataset=="T81" and param_clf__n_estimators==1000 and param_clf__max_depth==10 {s_brt}'
dfTmp = deepcopy(dfRes.query(s_query)).sort_values('balanced_accuracy',ascending=False)
print(s_query)
print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")
# print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")

dfTmp[lPrettyCols]

MainDataset=="T81" and param_clf__n_estimators==1000 and param_clf__max_depth==10 and (RunType=="Boruta_True" or RunType=="Boruta_Top")
Max balanced accuracy 0.9346908208996904
Max balanced accuracy 0.9346908208996904


,MainDataset,RunType,classifier,brt_nTrue,brt_nTop,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md
169,T81,Boruta_True,RandomForestClassifier,26.0,30.0,0.937009,0.934691,0.968085,0.928179,0.912577,0.912577,0.956805,0.944785,0.931586,0.944785,5.0
251,T81,Boruta_True,RandomForestClassifier,25.0,29.0,0.937009,0.934691,0.967777,0.928179,0.912577,0.912577,0.956805,0.944785,0.931586,0.944785,7.0
210,T81,Boruta_Top,RandomForestClassifier,26.0,30.0,0.937015,0.934681,0.968502,0.928258,0.912577,0.912577,0.956785,0.944772,0.931428,0.944772,5.0
87,T81,Boruta_True,RandomForestClassifier,29.0,34.0,0.935917,0.933471,0.970090,0.926729,0.910138,0.910138,0.956805,0.944690,0.930082,0.944690,3.0
292,T81,Boruta_Top,RandomForestClassifier,25.0,29.0,0.935922,0.933022,0.969853,0.926415,0.905319,0.905319,0.960726,0.949371,0.926811,0.949371,7.0
128,T81,Boruta_Top,RandomForestClassifier,29.0,34.0,0.932662,0.929818,0.969575,0.922771,0.902850,0.902850,0.956785,0.944158,0.924656,0.944158,3.0
139,T81,Boruta_Top,XGBClassifier,29.0,34.0,0.926146,0.922540,0.961511,0.914641,0.888275,0.888275,0.956805,0.944143,0.914665,0.944143,3.0
149,T81,Boruta_Top,XGBClassifier,29.0,34.0,0.923960,0.920101,0.958472,0.911684,0.883397,0.883397,0.956805,0.943478,0.911513,0.943478,3.0
303,T81,Boruta_Top,XGBClassifier,25.0,29.0,0.919630,0.916418,0.954665,0.907801,0.885836,0.885836,0.947001,0.931853,0.911655,0.931853,7.0
272,T81,Boruta_True,XGBClassifier,25.0,29.0,0.919624,0.916194,0.952037,0.907482,0.883426,0.883426,0.948961,0.933728,0.910132,0.933728,7.0


In [20]:
# dfTmp.query('classifier=="RandomForestClassifier" and (RunType=="Boruta_True" or RunType=="Boruta_Top")')
dfTmp.query(
    'classifier=="RandomForestClassifier"'
)

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
169,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_True,5,26.0,30.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.937009,0.934691,0.968085,0.928179,0.912577,0.912577,0.956805,0.944785,0.931586,0.944785,5.0,T1000_itrr500_th100_topR5
251,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_True,5,25.0,29.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.937009,0.934691,0.967777,0.928179,0.912577,0.912577,0.956805,0.944785,0.931586,0.944785,7.0,T1000_itrr500_th100_topR5
210,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_Top,5,26.0,30.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.937015,0.934681,0.968502,0.928258,0.912577,0.912577,0.956785,0.944772,0.931428,0.944772,5.0,T1000_itrr500_th100_topR5
87,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_True,5,29.0,34.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.935917,0.933471,0.970090,0.926729,0.910138,0.910138,0.956805,0.944690,0.930082,0.944690,3.0,T1000_itrr500_th100_topR5
292,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_Top,5,25.0,29.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.935922,0.933022,0.969853,0.926415,0.905319,0.905319,0.960726,0.949371,0.926811,0.949371,7.0,T1000_itrr500_th100_topR5
128,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,RandomForestClassifier,Boruta_Top,5,29.0,34.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.932662,0.929818,0.969575,0.922771,0.902850,0.902850,0.956785,0.944158,0.924656,0.944158,3.0,T1000_itrr500_th100_topR5


In [21]:
dfTmp.query('classifier=="XGBClassifier"')

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
139,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_Top,5,29.0,34.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.926146,0.922540,0.961511,0.914641,0.888275,0.888275,0.956805,0.944143,0.914665,0.944143,3.0,T1000_itrr500_th100_topR5
149,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_Top,5,29.0,34.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.923960,0.920101,0.958472,0.911684,0.883397,0.883397,0.956805,0.943478,0.911513,0.943478,3.0,T1000_itrr500_th100_topR5
303,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_Top,5,25.0,29.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.919630,0.916418,0.954665,0.907801,0.885836,0.885836,0.947001,0.931853,0.911655,0.931853,7.0,T1000_itrr500_th100_topR5
272,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,25.0,29.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.919624,0.916194,0.952037,0.907482,0.883426,0.883426,0.948961,0.933728,0.910132,0.933728,7.0,T1000_itrr500_th100_topR5
262,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,25.0,29.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.919630,0.915969,0.956227,0.907317,0.881017,0.881017,0.950922,0.935892,0.908450,0.935892,7.0,T1000_itrr500_th100_topR5
98,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,29.0,34.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.919630,0.915725,0.957389,0.906942,0.878548,0.878548,0.952902,0.938507,0.907318,0.938507,3.0,T1000_itrr500_th100_topR5
108,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,29.0,34.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.918537,0.914511,0.953515,0.905444,0.876139,0.876139,0.952883,0.938079,0.905744,0.938079,3.0,T1000_itrr500_th100_topR5
180,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,26.0,30.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.916363,0.913243,0.955692,0.904225,0.883426,0.883426,0.943060,0.926785,0.909444,0.926785,5.0,T1000_itrr500_th100_topR5
313,T81,T81_Boruta_Top,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_Top,5,25.0,29.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.916375,0.912814,0.951718,0.903577,0.878607,0.878607,0.947020,0.931722,0.906948,0.931722,7.0,T1000_itrr500_th100_topR5
190,T81,T81_Boruta_True,2024-04-27 21:48:30.513965,XGBClassifier,Boruta_True,5,26.0,30.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.916357,0.912784,0.952876,0.903537,0.878548,0.878548,0.947020,0.931953,0.907021,0.931953,5.0,T1000_itrr500_th100_topR5


In [91]:
dfTmp.crosstab('RunType','mean_test_balanced_accuracy')

AttributeError: 'DataFrame' object has no attribute 'crosstab'

# Inspect DT Dataset
- notes: based on the results below - T85 Model is a bust.. looking into the acutal selected features, the T81 model has a lot of symptoms that are much more important

In [115]:
dfRes.query(
    'MainDataset=="DT"').sort_values('balanced_accuracy',ascending=False)

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
384,DT,DT_Full,2024-04-27 20:19:58.905855,XGBClassifier,Full,5,NaN,NaN,"XGBClassifier(base_score=None, booster=None, c...",10.0,100.0,42.0,NaN,0.1,0.969430,0.968657,0.992039,0.967185,0.954968,0.954968,0.982347,0.979750,0.960689,0.979750,NaN,T1000_itrr500_th100_topR5
377,DT,DT_Full,2024-04-27 20:19:58.905855,RandomForestClassifier,Full,5,NaN,NaN,RandomForestClassifier(),NaN,100.0,42.0,NaN,NaN,0.968653,0.968187,0.991759,0.966553,0.959914,0.959914,0.976460,0.973294,0.964630,0.973294,NaN,T1000_itrr500_th100_topR5
394,DT,DT_Full,2024-04-27 20:19:58.905855,XGBClassifier,Full,5,NaN,NaN,"XGBClassifier(base_score=None, booster=None, c...",10.0,100.0,42.0,NaN,0.2,0.968653,0.968039,0.991944,0.966462,0.957167,0.957167,0.978912,0.975958,0.962391,0.975958,NaN,T1000_itrr500_th100_topR5
378,DT,DT_Full,2024-04-27 20:19:58.905855,RandomForestClassifier,Full,5,NaN,NaN,RandomForestClassifier(),NaN,1000.0,42.0,NaN,NaN,0.968394,0.967942,0.992465,0.966285,0.959914,0.959914,0.975970,0.972750,0.964611,0.972750,NaN,T1000_itrr500_th100_topR5
382,DT,DT_Full,2024-04-27 20:19:58.905855,XGBClassifier,Full,5,NaN,NaN,"XGBClassifier(base_score=None, booster=None, c...",6.0,100.0,42.0,NaN,0.1,0.968394,0.967500,0.991088,0.966000,0.951672,0.951672,0.983329,0.980778,0.957956,0.980778,NaN,T1000_itrr500_th100_topR5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,DT,DT_Boruta_Top,2024-04-27 20:19:58.905855,RandomForestClassifier,Boruta_Top,5,30.0,35.0,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.897210,0.891883,0.982449,0.879681,0.800577,0.800577,0.983190,0.977453,0.847744,0.977453,7.0,T1000_itrr500_th100_topR5
369,DT,DT_Full,2024-04-27 20:19:58.905855,RandomForestClassifier,Full,5,NaN,NaN,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.876166,0.869398,0.985719,0.850828,0.749585,0.749585,0.989212,0.984102,0.815804,0.984102,NaN,T1000_itrr500_th100_topR5
370,DT,DT_Full,2024-04-27 20:19:58.905855,RandomForestClassifier,Full,5,NaN,NaN,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.871762,0.864672,0.987700,0.844396,0.739151,0.739151,0.990192,0.985359,0.809868,0.985359,NaN,T1000_itrr500_th100_topR5
328,DT,DT_Holdout,2024-04-27 20:19:58.905855,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.862743,0.855471,0.984084,0.832273,0.722991,0.722991,0.987952,0.982249,0.799760,0.982249,NaN,T1000_itrr500_th100_topR5


In [31]:
s_query = f'MainDataset=="DT" and param_clf__n_estimators==1000 and param_clf__max_depth==10 {s_brt}'
dfTmp = deepcopy(dfRes.query(s_query)).sort_values("balanced_accuracy", ascending=False)
print(s_query)
print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")
# print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")

dfTmp[lPrettyCols]

MainDataset=="DT" and param_clf__n_estimators==1000 and param_clf__max_depth==10 and (RunType=="Boruta_True" or RunType=="Boruta_Top")
Max balanced accuracy 0.9621018436681088


,MainDataset,RunType,classifier,brt_nTrue,brt_nTop,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md
825,DT,Boruta_True,RandomForestClassifier,35.0,39.0,0.963194,0.962102,0.985940,0.960162,0.943423,0.943423,0.980780,0.977911,0.951517,0.977911,5.0
907,DT,Boruta_True,RandomForestClassifier,30.0,35.0,0.963194,0.962102,0.984675,0.960162,0.943423,0.943423,0.980780,0.977911,0.951517,0.977911,7.0
948,DT,Boruta_Top,RandomForestClassifier,30.0,35.0,0.961929,0.960769,0.985243,0.958727,0.940757,0.940757,0.980780,0.977833,0.949352,0.977833,7.0
743,DT,Boruta_True,RandomForestClassifier,40.0,45.0,0.960663,0.959417,0.986893,0.957270,0.938054,0.938054,0.980780,0.977791,0.947233,0.977791,3.0
866,DT,Boruta_Top,RandomForestClassifier,35.0,39.0,0.960663,0.959417,0.986719,0.957270,0.938054,0.938054,0.980780,0.977791,0.947233,0.977791,5.0
784,DT,Boruta_Top,RandomForestClassifier,40.0,45.0,0.960663,0.959417,0.987260,0.957270,0.938054,0.938054,0.980780,0.977791,0.947233,0.977791,3.0
918,DT,Boruta_True,XGBClassifier,30.0,35.0,0.932758,0.931879,0.965310,0.927350,0.916541,0.916541,0.947217,0.940183,0.928913,0.940183,7.0
836,DT,Boruta_True,XGBClassifier,35.0,39.0,0.931484,0.930385,0.964780,0.925845,0.911171,0.911171,0.949598,0.942633,0.924380,0.942633,5.0
795,DT,Boruta_Top,XGBClassifier,40.0,45.0,0.928937,0.927825,0.966396,0.923176,0.908432,0.908432,0.947217,0.939518,0.921591,0.939518,3.0
754,DT,Boruta_True,XGBClassifier,40.0,45.0,0.928937,0.927646,0.967266,0.922841,0.905694,0.905694,0.949598,0.941929,0.919861,0.941929,3.0


In [33]:
s_query = f'MainDataset=="DT" and param_clf__n_estimators==1000 and param_clf__max_depth==6 {s_brt}'
dfTmp = deepcopy(dfRes.query(s_query)).sort_values("balanced_accuracy", ascending=False)
print(s_query)
print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")
# print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")

dfTmp[lPrettyCols]

MainDataset=="DT" and param_clf__n_estimators==1000 and param_clf__max_depth==6 and (RunType=="Boruta_True" or RunType=="Boruta_Top")
Max balanced accuracy 0.95536310492937


,MainDataset,RunType,classifier,brt_nTrue,brt_nTop,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md
864,DT,Boruta_Top,RandomForestClassifier,35.0,39.0,0.956857,0.955363,0.985663,0.952897,0.929946,0.929946,0.980780,0.977597,0.940782,0.977597,5.0
741,DT,Boruta_True,RandomForestClassifier,40.0,45.0,0.955599,0.954030,0.985922,0.951319,0.927279,0.927279,0.980780,0.977579,0.939035,0.977579,3.0
905,DT,Boruta_True,RandomForestClassifier,30.0,35.0,0.955599,0.954012,0.984480,0.951233,0.927243,0.927243,0.980780,0.977613,0.939210,0.977613,7.0
823,DT,Boruta_True,RandomForestClassifier,35.0,39.0,0.955599,0.954012,0.986132,0.951233,0.927243,0.927243,0.980780,0.977613,0.939210,0.977613,5.0
946,DT,Boruta_Top,RandomForestClassifier,30.0,35.0,0.955599,0.954012,0.985209,0.951357,0.927243,0.927243,0.980780,0.977620,0.938932,0.977620,7.0
782,DT,Boruta_Top,RandomForestClassifier,40.0,45.0,0.944183,0.941886,0.986370,0.937926,0.902991,0.902991,0.980780,0.977078,0.920389,0.977078,3.0
834,DT,Boruta_True,XGBClassifier,35.0,39.0,0.928953,0.927875,0.965421,0.922954,0.908505,0.908505,0.947246,0.939580,0.922289,0.939580,5.0
793,DT,Boruta_Top,XGBClassifier,40.0,45.0,0.928928,0.927678,0.966707,0.922836,0.905730,0.905730,0.949627,0.941764,0.919806,0.941764,3.0
916,DT,Boruta_True,XGBClassifier,30.0,35.0,0.927695,0.926510,0.964813,0.921305,0.905802,0.905802,0.947217,0.939608,0.920761,0.939608,7.0
957,DT,Boruta_Top,XGBClassifier,30.0,35.0,0.925163,0.923854,0.965062,0.918344,0.900432,0.900432,0.947275,0.939481,0.916532,0.939481,7.0


In [136]:
dfTmp = deepcopy(dfRes.query(
    f'MainDataset=="DT" and param_clf__n_estimators==1000 and param_clf__max_depth==15 {s_brt}'
)).sort_values('balanced_accuracy',ascending=False)
dfTmp

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
417,DT,DT_Boruta_True,2024-04-27 20:19:58.905855,RandomForestClassifier,Boruta_True,5,30.0,35.0,RandomForestClassifier(),15.0,1000.0,42.0,NaN,NaN,0.965726,0.964787,0.985329,0.963014,0.948793,0.948793,0.980780,0.978027,0.955902,0.978027,7.0,T1000_itrr500_th100_topR5
458,DT,DT_Boruta_Top,2024-04-27 20:19:58.905855,RandomForestClassifier,Boruta_Top,5,30.0,35.0,RandomForestClassifier(),15.0,1000.0,42.0,NaN,NaN,0.964460,0.963453,0.987075,0.961599,0.946126,0.946126,0.980780,0.977951,0.953686,0.977951,7.0,T1000_itrr500_th100_topR5
428,DT,DT_Boruta_True,2024-04-27 20:19:58.905855,XGBClassifier,Boruta_True,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,0.1,0.931484,0.930513,0.964216,0.925862,0.913838,0.913838,0.947189,0.940706,0.927168,0.940706,7.0,T1000_itrr500_th100_topR5
479,DT,DT_Boruta_Top,2024-04-27 20:19:58.905855,XGBClassifier,Boruta_Top,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,0.2,0.927687,0.926670,0.963582,0.921693,0.908505,0.908505,0.944836,0.936770,0.922020,0.936770,7.0,T1000_itrr500_th100_topR5
438,DT,DT_Boruta_True,2024-04-27 20:19:58.905855,XGBClassifier,Boruta_True,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,0.2,0.926413,0.925609,0.962887,0.920689,0.911171,0.911171,0.940046,0.932258,0.923990,0.932258,7.0,T1000_itrr500_th100_topR5
469,DT,DT_Boruta_Top,2024-04-27 20:19:58.905855,XGBClassifier,Boruta_Top,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,0.1,0.926421,0.925012,0.965358,0.919828,0.900396,0.900396,0.949627,0.942258,0.916167,0.942258,7.0,T1000_itrr500_th100_topR5
489,DT,DT_Boruta_Top,2024-04-27 20:19:58.905855,XGBClassifier,Boruta_Top,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.916270,0.914834,0.953706,0.908736,0.889622,0.889622,0.940046,0.930293,0.906774,0.930293,7.0,T1000_itrr500_th100_topR5
448,DT,DT_Boruta_True,2024-04-27 20:19:58.905855,XGBClassifier,Boruta_True,5,30.0,35.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.915004,0.913804,0.955818,0.907777,0.892324,0.892324,0.935284,0.925756,0.908605,0.925756,7.0,T1000_itrr500_th100_topR5


In [130]:
dfRes['param_clf__max_depth'].value_counts(dropna=False)

param_clf__max_depth
NaN     108
3.0      96
6.0      96
10.0     96
15.0     96
Name: count, dtype: int64

# Inspect T85 Dataset
- notes: based on the results below - T85 Model is a bust.. looking into the acutal selected features, the T81 model has a lot of symptoms that are much more important

In [113]:
dfRes.query(
    'MainDataset=="T85"').sort_values('balanced_accuracy',ascending=False)

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
174,T85,T85_Holdout,2024-04-27 20:03:55.582372,LogisticRegression,Holdout,5,NaN,NaN,LogisticRegression(),NaN,NaN,NaN,1000.0,NaN,0.678213,0.646719,0.701009,0.546889,0.486867,0.486867,0.806570,0.625576,0.701707,0.625576,NaN,T1000_itrr500_th100_topR5
217,T85,T85_Full,2024-04-27 20:03:55.582372,XGBClassifier,Full,5,NaN,NaN,"XGBClassifier(base_score=None, booster=None, c...",3.0,1000.0,42.0,NaN,0.1,0.677085,0.645026,0.719961,0.545313,0.484679,0.484679,0.805373,0.624194,0.701123,0.624194,NaN,T1000_itrr500_th100_topR5
215,T85,T85_Full,2024-04-27 20:03:55.582372,LogisticRegression,Full,5,NaN,NaN,LogisticRegression(),NaN,NaN,NaN,1000.0,NaN,0.680729,0.643743,0.716797,0.534732,0.458789,0.458789,0.828697,0.641446,0.696692,0.641446,NaN,T1000_itrr500_th100_topR5
228,T85,T85_Full,2024-04-27 20:03:55.582372,XGBClassifier,Full,5,NaN,NaN,"XGBClassifier(base_score=None, booster=None, c...",6.0,100.0,42.0,NaN,0.2,0.677661,0.643426,0.720470,0.539191,0.472214,0.472214,0.814637,0.629460,0.698543,0.629460,NaN,T1000_itrr500_th100_topR5
234,T85,T85_Full,2024-04-27 20:03:55.582372,XGBClassifier,Full,5,NaN,NaN,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,0.2,0.677661,0.643426,0.720470,0.539191,0.472214,0.472214,0.814637,0.629460,0.698543,0.629460,NaN,T1000_itrr500_th100_topR5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,T85,T85_Boruta_True,2024-04-27 20:03:55.582372,XGBClassifier,Boruta_True,5,2.0,6.0,"XGBClassifier(base_score=None, booster=None, c...",6.0,100.0,42.0,NaN,1.0,0.575277,0.524439,0.538896,0.336098,0.267579,0.267579,0.781299,0.458557,0.613768,0.458557,7.0,T1000_itrr500_th100_topR5
285,T85,T85_Boruta_True,2024-04-27 20:03:55.582372,XGBClassifier,Boruta_True,5,2.0,6.0,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,1.0,0.575277,0.524439,0.538896,0.336098,0.267579,0.267579,0.781299,0.458557,0.613768,0.458557,7.0,T1000_itrr500_th100_topR5
284,T85,T85_Boruta_True,2024-04-27 20:03:55.582372,XGBClassifier,Boruta_True,5,2.0,6.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.575281,0.524044,0.538336,0.334174,0.265226,0.265226,0.782862,0.459571,0.613425,0.459571,7.0,T1000_itrr500_th100_topR5
274,T85,T85_Boruta_True,2024-04-27 20:03:55.582372,XGBClassifier,Boruta_True,5,2.0,6.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,0.2,0.574338,0.524013,0.538612,0.337294,0.269877,0.269877,0.778150,0.456931,0.613604,0.456931,7.0,T1000_itrr500_th100_topR5


In [29]:
s_query = f'MainDataset=="T85" and param_clf__n_estimators==1000 and param_clf__max_depth==10 {s_brt}'
dfTmp = deepcopy(dfRes.query(s_query)).sort_values("balanced_accuracy", ascending=False)
print(s_query)
print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")

dfTmp[lPrettyCols]

MainDataset=="T85" and param_clf__n_estimators==1000 and param_clf__max_depth==10 and (RunType=="Boruta_True" or RunType=="Boruta_Top")
Max balanced accuracy 0.6155159614431747
Max balanced accuracy 0.6155159614431747


,MainDataset,RunType,classifier,brt_nTrue,brt_nTop,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md
456,T85,Boruta_Top,RandomForestClassifier,13.0,17.0,0.664784,0.615516,0.657380,0.466079,0.366293,0.366293,0.864739,0.642755,0.671020,0.642755,3.0
415,T85,Boruta_True,RandomForestClassifier,13.0,17.0,0.653455,0.609140,0.648108,0.471168,0.385062,0.385062,0.833219,0.608827,0.669219,0.608827,3.0
538,T85,Boruta_Top,RandomForestClassifier,6.0,10.0,0.648782,0.603234,0.633828,0.459205,0.373187,0.373187,0.833280,0.600771,0.665131,0.600771,5.0
436,T85,Boruta_True,XGBClassifier,13.0,17.0,0.596040,0.577830,0.593347,0.489775,0.485828,0.485828,0.669833,0.495098,0.661252,0.495098,3.0
467,T85,Boruta_Top,XGBClassifier,13.0,17.0,0.596984,0.574070,0.617891,0.476106,0.457811,0.457811,0.690330,0.497931,0.655518,0.497931,3.0
477,T85,Boruta_Top,XGBClassifier,13.0,17.0,0.595093,0.573605,0.610712,0.477988,0.464815,0.464815,0.682394,0.492634,0.656583,0.492634,3.0
487,T85,Boruta_Top,XGBClassifier,13.0,17.0,0.590367,0.570449,0.605741,0.476789,0.469466,0.469466,0.671432,0.486788,0.655108,0.486788,3.0
446,T85,Boruta_True,XGBClassifier,13.0,17.0,0.587572,0.568838,0.583735,0.477822,0.474118,0.474118,0.663558,0.483214,0.654408,0.483214,3.0
426,T85,Boruta_True,XGBClassifier,13.0,17.0,0.586629,0.567260,0.599053,0.475123,0.469412,0.469412,0.665108,0.482367,0.652660,0.482367,3.0
549,T85,Boruta_Top,XGBClassifier,6.0,10.0,0.578120,0.553217,0.590564,0.447082,0.427250,0.427250,0.679183,0.469850,0.639662,0.469850,5.0


# Difference between Boruta Selected Datasets

In [45]:
lDT_7 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV2/DT/FS_DT_Boruta_T1000_itrr500_th100_topR5_MD7.csv"
)

lR81_5 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV2/T81/FS_T81_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
)

lR81_7 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV2/T81/FS_T81_Boruta_T1000_itrr500_th100_topR5_MD7.csv"
)

len(lDT_7), len(lR81_5), len(lR81_7)

(30, 26, 25)

In [39]:
set(lDT_7).difference(set(lR81_5))

{'SYMPT-ear_pain___2',
 'SYMPT-runny_nose_rhinorrhea___1',
 'SYMPT-runny_nose_rhinorrhea___2',
 'SYMPT-weight_loss___1',
 'SYMPT-weight_loss___2',
 'SYMPT-wheezing_or_stridor___2'}

In [42]:
set(lR81_5).difference(set(lDT_7))

{'SYMPT-fever_38_0oc___1', 'SYMPT-fever_38_0oc___2'}

In [44]:
set(lR81_7).difference(set(lDT_7))

{'SYMPT-fever_38_0oc___2'}

In [41]:
set(lR81_5).difference(set(lR81_7))

{'SYMPT-fever_38_0oc___1'}

# Check Patient Distribution of DT

In [65]:
d = load_dataset_to_df(f"../Data/DataV2/featuresDT.arrow", verbose=True)

In [66]:
site_name_dict = {1: "LDN", 2: "MTL", 3: "SAN", 4: "RIO", 5: "LUS", 6: "CA", 7: "NA"}

In [67]:
d['sitecode'] = d['SITE'].replace(site_name_dict)

In [69]:
d = load_dataset_to_df(f"../Data/DataV2/featuresThresAll81.arrow", verbose=True)

In [70]:
site_name_dict = {1: "LDN", 2: "MTL", 3: "SAN", 4: "RIO", 5: "LUS", 6: "CA", 7: "NA"}

In [71]:
d['sitecode'] = d['SITE'].replace(site_name_dict)

In [68]:
print('DT Dataset')
pd.crosstab(d.LC_STATUS,d.sitecode)

DT Dataset


sitecode,LDN,LUS,MTL,RIO,SAN
LC_STATUS,,,,,
HC,16,38,0,9,2
LC_NEG,40,5,1962,7,65
LC_POS,28,3,909,650,266


In [72]:
print("T81 Dataset")
pd.crosstab(d.LC_STATUS,d.sitecode)

T81 Dataset


sitecode,LDN,LUS,MTL,RIO,SAN
LC_STATUS,,,,,
HC,32,38,0,9,3
LC_NEG,172,298,1997,7,71
LC_POS,33,199,916,650,266


# Validation Result

- lR81_5 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV2/T81/FS_T81_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
)


- lColFS = lR81_5
- dataName = "T81"
- selFeatName = "FS_T81_Boruta_T1000_itrr500_th100_topR5_MD5"

In [13]:
dfValRes = pd.read_csv(
    "OUTPUT/MP/05-classifiers/DataV2/T81/Val_T81_HoldoutVal_FS_T81_Boruta_T1000_itrr500_th100_topR5_MD5_SelCol.csv"
)
dfValRes.columns = [x.replace("mean_test_", "") for x in dfValRes.columns]

In [14]:
dfValRes

,MainDataset,Title,date,classifier,RunType,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV
0,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.868766,0.857840,0.966415,0.837058,0.753012,0.753012,0.962667,0.942443,0.827802,0.942443
1,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.872009,0.861693,0.967101,0.842143,0.762680,0.762680,0.960707,0.940397,0.833154,0.940397
2,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,0.924078,0.920751,0.969556,0.912731,0.888657,0.888657,0.952844,0.938699,0.913850,0.938699
3,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),6.0,1000.0,42.0,NaN,NaN,0.930599,0.927809,0.969316,0.920669,0.900793,0.900793,0.954824,0.941957,0.922611,0.941957
4,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),10.0,100.0,42.0,NaN,NaN,0.932761,0.931136,0.968815,0.924164,0.915310,0.915310,0.946962,0.933378,0.932472,0.933378
5,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.931675,0.929916,0.969820,0.922847,0.912871,0.912871,0.946962,0.933196,0.930646,0.933196
6,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),15.0,100.0,42.0,NaN,NaN,0.937098,0.935985,0.968671,0.929351,0.925007,0.925007,0.946962,0.934028,0.939858,0.934028
7,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),15.0,1000.0,42.0,NaN,NaN,0.937098,0.935970,0.969775,0.929335,0.924978,0.924978,0.946962,0.933998,0.939858,0.933998
8,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),NaN,100.0,42.0,NaN,NaN,0.933854,0.933497,0.963285,0.926349,0.929856,0.929856,0.937138,0.923044,0.942928,0.923044
9,T81,T81_HoldoutVal,2024-04-27 23:09:23.579472,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),NaN,1000.0,42.0,NaN,NaN,0.932767,0.932263,0.963869,0.925016,0.927388,0.927388,0.937138,0.922859,0.941118,0.922859


# Validation Results -- Final?

In [16]:
dfValRes.query(f'classifier=="RandomForestClassifier" and param_clf__max_depth==10 and param_clf__n_estimators==1000')[[x for x in lPrettyCols if 'brt_' not in x]].T

,5
MainDataset,T81
RunType,HoldoutVal
classifier,RandomForestClassifier
accuracy,0.931675
balanced_accuracy,0.929916
roc_auc,0.96982
f1,0.922847
recall,0.912871
sensitivity,0.912871
specificity,0.946962


# Validation Result - BY SITE

- lR81_5 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV2/T81/FS_T81_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
)


- lColFS = lR81_5
- dataName = "T81"
- selFeatName = "FS_T81_Boruta_T1000_itrr500_th100_topR5_MD5"

In [18]:
dfSite = pd.read_csv(
    "OUTPUT/MP/05-classifiers/DataV2/CLFRunCombined_Site_SelCol.csv"
)
dfSite.columns = [x.replace("mean_test_", "") for x in dfSite.columns]

In [19]:
# dfSite.query(s_RFXGB10)[lPrettyColsSite] # XGB doesnt help
dfSite.query(s_RF10)[lPrettyColsSite].T

,5,46,87,128,169,210,251
MainDataset,T81,T81,T81,T81,T81,T81,T81
RunType,HoldoutVal_Site,HoldoutVal_Site,HoldoutVal_Site,HoldoutVal_Site,HoldoutVal_Site,HoldoutVal_Site,HoldoutVal_Site
Site,LDN,MTL,SAN,RIO,LUS,CA,NorthA
classifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier
accuracy,0.9,0.991394,0.868132,0.9849,0.71,0.982348,0.976832
balanced_accuracy,0.735714,0.986186,0.796061,0.892308,0.675,0.978437,0.976454
roc_auc,0.971429,0.997222,0.895758,NaN,0.754167,0.993002,0.987935
f1,0.533333,0.985832,0.917905,0.992305,0.570696,0.970868,0.967417
recall,0.5,0.972372,0.925455,0.992308,0.5,0.968421,0.975255
sensitivity,0.5,0.972372,0.925455,NaN,0.5,0.968421,0.975255


In [ ]:
print("T81 Dataset")
pd.crosstab(d.LC_STATUS,d.sitecode)

T81 Dataset


sitecode,LDN,LUS,MTL,RIO,SAN
LC_STATUS,,,,,
HC,32,38,0,9,3
LC_NEG,172,298,1997,7,71
LC_POS,33,199,916,650,266
